<a href="https://colab.research.google.com/github/shubhamgundawarNYU/Big-Data-Project-Group-16/blob/main/NYPD_Complaint_Data_Historic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**BIG DATA PROJECT**

### NYPD COMPLAINTS DATA HISTORIC 
Link to Dataset (https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i)

### DATA CLEANING AT SCALE

#### Mounting Google Drive to Google Collab Notebook to Load the Data Set

Make sure you have the dataset in your Google Drive and you mount your drive to the Colab.

The file should be at the following path: `gdrive/My Drive/NYPD_Complaint_Data_Historic.csv`


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Importing required and Necessary Libraries for cleaning the data present in the data set






In [ ]:
import numpy as np
import pandas as pd
import io

In [ ]:
!pip install pyspark

#### Installing and importing openclean python library for data profiling and statistical inference

In [ ]:
pip install openclean humanfriendly

     |████████████████████████████████| 86 kB 3.2 MB/s 
     |████████████████████████████████| 267 kB 36.5 MB/s 
     |████████████████████████████████| 137 kB 57.0 MB/s 
     |████████████████████████████████| 69 kB 6.3 MB/s 
     |████████████████████████████████| 109 kB 72.2 MB/s 
     |████████████████████████████████| 260 kB 62.8 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 206 kB 52.0 MB/s 
     |████████████████████████████████| 525 kB 74.8 MB/s 
     |████████████████████████████████| 180 kB 58.8 MB/s 
     |████████████████████████████████| 149 kB 67.5 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 3.6 MB 75.4 MB/s 
     |████████████████████████████████| 961 kB 42.9 MB/s 
     |████████████████████████████████| 636 kB 73.6 MB/s 
  Created wheel for datasize: filename=datasize-1.0.0-py2.py3-none-any.whl size=


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

Run a local spark session to test your installation:

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

#### Reading the Data Set CSV File using `spark.read.csv()` Function

In [ ]:
df = spark.read.csv("/content/gdrive/MyDrive/NYPD_Complaint_Data_Historic.csv", inferSchema=True, header =True)

### In order to find similar datasets to 'NYPD Complaint Data Historic', we use openclean python library's easy integration with Socrata NYC Open Data engine and find out all the datasets that have similar column structure as our original dataset.

In [ ]:
import openclean
from openclean.data.source.socrata import Socrata

for dataset in Socrata().catalog(domain='data.cityofnewyork.us'):
    if 'complaint' in dataset.name.lower() or 'NYPD' in dataset.name or 'Crime' in dataset.name.lower():
        print(f'{dataset.identifier}\t{dataset.domain}\t{dataset.name}')

qgea-i56i	data.cityofnewyork.us	NYPD Complaint Data Historic
uip8-fykc	data.cityofnewyork.us	NYPD Arrest Data (Year to Date)
eabe-havv	data.cityofnewyork.us	DOB Complaints Received
5uac-w243	data.cityofnewyork.us	NYPD Complaint Data Current (Year To Date)
8h9b-rp9u	data.cityofnewyork.us	NYPD Arrests Data (Historic)
833y-fsy8	data.cityofnewyork.us	NYPD Shooting Incident Data (Historic)
5ucz-vwe8	data.cityofnewyork.us	NYPD Shooting Incident Data (Year To Date)
uwyv-629c	data.cityofnewyork.us	Housing Maintenance Code Complaints
a2nx-4u46	data.cityofnewyork.us	Complaint Problems
bqiq-cu78	data.cityofnewyork.us	NYPD Hate Crimes
sv2w-rv3k	data.cityofnewyork.us	NYPD Criminal Court Summons (Historic)
nre2-6m2s	data.cityofnewyork.us	Consumer Services Mediated Complaints
mv4k-y93f	data.cityofnewyork.us	NYPD Criminal Court Summons Incident Level Data (Year To Date)
6v9u-ndjg	data.cityofnewyork.us	Building Complaint Disposition Codes
9jgj-bmct	data.cityofnewyork.us	DOHMH Indoor Environmental Compl

#### From the above list we select 10 datasets that have maximum overlapping columns with 'NYPD Complaint Data Historic'

# **Data Profiling**

In [ ]:
df.count()

7375993

#### Get Data Type for each column present in the Data Set




In [ ]:
df.printSchema()

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- CMPLNT_TO_DT: string (nullable = true)
 |-- CMPLNT_TO_TM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- HOUSING_PSA: string (nullable = true)
 |-- X_COORD_CD: integer (nullable = true)
 |-- Y_COORD_CD: integer (nullable = true)
 |-- SUSP_AG

#### Outputing the List of Columns in the Data Set

In [ ]:
df.columns

['CMPLNT_NUM',
 'CMPLNT_FR_DT',
 'CMPLNT_FR_TM',
 'CMPLNT_TO_DT',
 'CMPLNT_TO_TM',
 'ADDR_PCT_CD',
 'RPT_DT',
 'KY_CD',
 'OFNS_DESC',
 'PD_CD',
 'PD_DESC',
 'CRM_ATPT_CPTD_CD',
 'LAW_CAT_CD',
 'BORO_NM',
 'LOC_OF_OCCUR_DESC',
 'PREM_TYP_DESC',
 'JURIS_DESC',
 'JURISDICTION_CODE',
 'PARKS_NM',
 'HADEVELOPT',
 'HOUSING_PSA',
 'X_COORD_CD',
 'Y_COORD_CD',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'TRANSIT_DISTRICT',
 'Latitude',
 'Longitude',
 'Lat_Lon',
 'PATROL_BORO',
 'STATION_NAME',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX']

#### Get top 10 rows of the complaints dataframe

In [ ]:
df.show(n=10)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+----------------+-----------------+--------+----------+-----------+----------+----------+--------------+---------+--------+----------------+------------------+------------------+--------------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|          Latitude|         Longitude|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+--------

Check for duplicate values in dataset

In [ ]:
df.distinct().count()

7375993

In [ ]:
# Download the full 'NYPD Complaint Data Historic' dataset.
# Note that the downloaded full dataset file is about 380 MB in size! Use the
# alternative data file with 10,000 rows that is included in the repository if
# you do not want to download the full data file.

import gzip
import humanfriendly
import os

dataset = Socrata().dataset('qgea-i56i')

# By default, this example uses a small sample of the full dataset that
# is included in the 'data' subfolder within this repository.
#datafile = './data/qgea-i56i.tsv.gz'

# Remove the comment for this line if you want to use the full dataset.
datafile = './qgea-i56i.tsv.gz'


# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)


fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using 'NYPD Complaint Data Historic' in file ./qgea-i56i.tsv.gz of size 475.48 MB


In [ ]:
# Due to the size of the full dataset file, we make use of openclean's
# stream operator to avoid having to load the dataset into main-memory.

from openclean.pipeline import stream

ds = stream(datafile)

In [ ]:
# Profile the resulting dataset view using the default data profiler.

from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.profile(default_profiler=DefaultColumnProfiler)

In [ ]:
# Print overview of profiling results.

profiles.stats()

,total,empty,distinct,uniqueness,entropy
CMPLNT_NUM,7375993,0,7373143,9.996136e-01,22.813633
CMPLNT_FR_DT,7375993,655,8606,1.166862e-03,12.425578
CMPLNT_FR_TM,7375993,48,1441,1.953648e-04,8.136466
CMPLNT_TO_DT,7375993,1704204,6825,1.203324e-03,12.417984
CMPLNT_TO_TM,7375993,1699541,1441,2.538558e-04,8.862856
ADDR_PCT_CD,7375993,2166,77,1.044234e-05,6.148690
RPT_DT,7375993,0,5479,7.428152e-04,12.405384
KY_CD,7375993,0,74,1.003255e-05,4.170727
OFNS_DESC,7375993,18823,71,9.650450e-06,4.006583
PD_CD,7375993,6278,432,5.861828e-05,5.913459


# **Data Cleaning**

## We see that the columns `X_COORD_CD`,`Y_COORD_CD`,`Latitude`, `Longitude` conveys the same data as `Lat_Lon`.

#### Hence, we drop those columns and keep only `Lat_Lon` column in our cleaned dataset.

In [ ]:
df = df.drop('X_COORD_CD','Y_COORD_CD','Latitude','Longitude')

In [ ]:
df.columns

['CMPLNT_NUM',
 'CMPLNT_FR_DT',
 'CMPLNT_FR_TM',
 'CMPLNT_TO_DT',
 'CMPLNT_TO_TM',
 'ADDR_PCT_CD',
 'RPT_DT',
 'KY_CD',
 'OFNS_DESC',
 'PD_CD',
 'PD_DESC',
 'CRM_ATPT_CPTD_CD',
 'LAW_CAT_CD',
 'BORO_NM',
 'LOC_OF_OCCUR_DESC',
 'PREM_TYP_DESC',
 'JURIS_DESC',
 'JURISDICTION_CODE',
 'PARKS_NM',
 'HADEVELOPT',
 'HOUSING_PSA',
 'SUSP_AGE_GROUP',
 'SUSP_RACE',
 'SUSP_SEX',
 'TRANSIT_DISTRICT',
 'Lat_Lon',
 'PATROL_BORO',
 'STATION_NAME',
 'VIC_AGE_GROUP',
 'VIC_RACE',
 'VIC_SEX']

#### Removing all the **duplicate** entries

In [ ]:
df = df.drop_duplicates()

#### **Checking** if the complaint number is unique or not

In [ ]:
df.select('CMPLNT_NUM').distinct().count()

7373143

#### As we can see `CMPLNT_NUM` should have been unique, but it is not.
#### Let's see what are the duplicate values.

In [ ]:
df1 = df.groupBy('CMPLNT_NUM').count().filter("count > 1")
df1.drop('count').count()

2850

In [ ]:
df1.sort('CMPLNT_NUM').show(n = 10)

+----------+-----+
|CMPLNT_NUM|count|
+----------+-----+
| 100490719|    2|
| 100509703|    2|
| 100519696|    2|
| 100757966|    2|
| 101031690|    2|
| 101337283|    2|
| 101364555|    2|
| 101422279|    2|
| 101500555|    2|
| 101897138|    2|
+----------+-----+
only showing top 10 rows



#### Check for complaint number `100509703`

In [ ]:
df.filter('CMPLNT_NUM = 100509703').show()

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+-------------+----------------+-----------------+--------+----------+-----------+--------------+--------------+--------+----------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+------

#### We understand, that complaint number is not specifically unique. The dataset has rows with duplicated complaint numbers having unique information for all other columns. Thus, we cannot drop the entries with duplicated complaint numbers.

## Find Count of Null, None, NaN of All DataFrame Columns

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-------

#### Get top 5 rows where complaint from Date is NaN

In [ ]:
df.where(col('CMPLNT_FR_DT').isNull()).show(n=5)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------+--------+----------------+--------------------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+-------------

### Get rows where either complaint from date or complaint from time is null

In [ ]:
df.filter(df.CMPLNT_FR_DT.isNull() | df.CMPLNT_FR_TM.isNull()).show(5)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+----------+----------+--------------+---------+--------+----------------+------------+-------------+--------------------+--------------------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|X_COORD_CD|Y_COORD_CD|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|    Latitude|    Longitude|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------

1. There maybe null values in the CMPLNT_TO_DT, CMPLNT_TO_TM, LOC_OF_OCCUR_DESC, PREM_TYP_DESC, PARKS_NM and HADEVELOPT columns. 

2. LOC_OF_OCCUR_DESC, PREM_TYP_DESC, PARKS_NM and HADEVELOPT need not be present for all the fields so cannot be used to eliminate records.

3. However, complaint from date and complaint from time columns cannot have null values. We drop the rows where either complaint from date or complaint from time is null.

In [ ]:
df = df.filter(df.CMPLNT_FR_DT.isNotNull() & df.CMPLNT_FR_TM.isNotNull())

In [ ]:
df.where(col('CMPLNT_FR_DT').isNull()).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|Lat_Lon|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+--------

In [ ]:
df.where(col('CMPLNT_TO_DT').isNull()).count()

1704046

Check if minimum and maximum values of date and time are valid or not. The value of time cannot be 24:00:00. 

In [ ]:
## Minimum value of the column in pyspark
df.agg({'CMPLNT_FR_TM': 'min'}).show()

+-----------------+
|min(CMPLNT_FR_TM)|
+-----------------+
|         00:00:00|
+-----------------+



In [ ]:
## Maximum value of the column in pyspark
df.agg({'CMPLNT_FR_TM': 'max'}).show()

+-----------------+
|max(CMPLNT_FR_TM)|
+-----------------+
|         23:59:00|
+-----------------+



In [ ]:
df.agg({'CMPLNT_FR_DT': 'min'}).show()

+-----------------+
|min(CMPLNT_FR_DT)|
+-----------------+
|       01/01/1948|
+-----------------+



In [ ]:
df.agg({'CMPLNT_FR_DT': 'max'}).show()

+-----------------+
|max(CMPLNT_FR_DT)|
+-----------------+
|       12/31/2020|
+-----------------+



In [ ]:
df.select('CMPLNT_FR_DT').distinct().orderBy('CMPLNT_FR_DT').show()

+------------+
|CMPLNT_FR_DT|
+------------+
|  01/01/1948|
|  01/01/1955|
|  01/01/1964|
|  01/01/1965|
|  01/01/1967|
|  01/01/1968|
|  01/01/1969|
|  01/01/1970|
|  01/01/1971|
|  01/01/1972|
|  01/01/1973|
|  01/01/1974|
|  01/01/1975|
|  01/01/1976|
|  01/01/1977|
|  01/01/1978|
|  01/01/1979|
|  01/01/1980|
|  01/01/1981|
|  01/01/1982|
+------------+
only showing top 20 rows



In [ ]:
from datetime import datetime
date_from = '2006-01-01'
date_to = '2020-12-31'

# Setting an user define function:
# This function converts the string cell into a date:
func =  udf (lambda x: datetime.strptime(str(x), '%m/%d/%Y'), DateType())

df = df.withColumn('CMPLNT_FR_DT', func(col('CMPLNT_FR_DT')))
df = df.filter(df.CMPLNT_FR_DT >= date_from).filter(df.CMPLNT_FR_DT <= date_to)
df.show()

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------+--------+----------------+--------------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+----

In [ ]:
df.select('CMPLNT_FR_DT').distinct().show()

+------------+
|CMPLNT_FR_DT|
+------------+
|  2019-06-04|
|  2019-05-08|
|  2016-03-01|
|  2014-11-12|
|  2018-05-28|
|  2020-08-24|
|  2018-08-10|
|  2019-11-18|
|  2019-11-01|
|  2006-04-23|
|  2019-09-22|
|  2011-05-06|
|  2017-01-06|
|  2016-04-25|
|  2018-06-06|
|  2018-03-17|
|  2020-01-21|
|  2019-11-21|
|  2018-09-01|
|  2016-05-03|
+------------+
only showing top 20 rows



In [ ]:
df.agg({'CMPLNT_FR_DT': 'min'}).show()

+-----------------+
|min(CMPLNT_FR_DT)|
+-----------------+
|       2006-01-01|
+-----------------+



In [ ]:
df.filter(df.CMPLNT_FR_DT < date_from).filter(df.CMPLNT_FR_DT > date_to).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|Lat_Lon|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+--------

In [ ]:
df.filter(df.CMPLNT_TO_DT < date_from).filter(df.CMPLNT_TO_DT > date_to).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-------

**Some basic data quality checks are as below:**
1. Check if there are no garbage values in the location of occurence description column. The valid values that this column should ideally contain are: 'FRONT OF', 'REAR OF', 'OUTSIDE', 'INSIDE', 'OPPOSITE OF'
2. Check if there are no garbage values in law category column. The valid values are: 'FELONY', 'VIOLATION', 'MISDEMEANOR'
3. Check if there are no misspellings in Borough Name. There should be 5 distinct boroughs: Manhattan, Bronx, Queens, Brooklyn, Staten Island. We implement unique method, in case of misspellings multiple values of the same borough would be returned.
4. Check if 'CRM_ATPT_CPTD_CD' column has no garbage value. The only acceptable values are Completed or Attempted.
5. Ideally, key code should contain only 3 digits. Implementing a check below to see if there are any invalid values for the key code. 

In [ ]:
df.select('LOC_OF_OCCUR_DESC').distinct().show()

+-----------------+
|LOC_OF_OCCUR_DESC|
+-----------------+
|      OPPOSITE OF|
|          REAR OF|
|             null|
|           INSIDE|
|          OUTSIDE|
|         FRONT OF|
+-----------------+



In [ ]:
df.select('LAW_CAT_CD').distinct().show()

+-----------+
| LAW_CAT_CD|
+-----------+
|     FELONY|
|MISDEMEANOR|
|  VIOLATION|
+-----------+



### Checks for Borough Name

In [ ]:
df.select('BORO_NM').distinct().show()

+-------------+
|      BORO_NM|
+-------------+
|         null|
|       QUEENS|
|     BROOKLYN|
|        BRONX|
|    MANHATTAN|
|STATEN ISLAND|
+-------------+



We can see there are no misspellings for the Borough names and thus no need for additional data correction for the same.

In [ ]:
df.where(col('BORO_NM').isNull()).show()

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+-------+----------------+----------+-------+-----------------+-------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------------+--------+----------------+-----------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|           SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+-------+----------------+----------+-------+-----------------+-------------+----------

In [ ]:
df = df.filter(df.BORO_NM.isNotNull())

In [ ]:
bor = ds.select('BORO_NM')
bor1 = bor.to_df()
bor1['BORO_NM'].replace('', np.nan, inplace=True)
bor1.dropna(inplace=True)

In [ ]:
bor1.isnull().values.any()

False

In [ ]:
bor1['BORO_NM'].unique()

array(['MANHATTAN', 'BRONX', 'QUEENS', 'BROOKLYN', 'STATEN ISLAND'],
      dtype=object)

In [ ]:
from openclean.function.matching.base import DefaultStringMatcher
from openclean.function.matching.fuzzy import FuzzySimilarity
from openclean.data.mapping import Mapping

VOCABULARY = ['BROOKLYN' ,'MANHATTAN','STATEN ISLAND','BRONX', 'QUEENS']

matcher = DefaultStringMatcher(
    vocabulary=VOCABULARY,
    similarity=FuzzySimilarity()
)

map = Mapping()
for query in bor1['BORO_NM']:
    map.add(query, matcher.find_matches(query))

print(map)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from openclean.function.eval.domain import Lookup
from openclean.operator.transform.update import update
from openclean.function.eval.base import Col

fixed = update(bor1, 'BORO_NM', Lookup(columns=['BORO_NM'], mapping=map.to_lookup(), default=Col('BORO_NM')))

print(fixed['BORO_NM'].unique())

/usr/local/lib/python3.7/dist-packages/openclean/data/mapping.py:223: UserWarning: Ignoring key: MANHATTAN (1771637 matches). To include ignored keys, update the map to contain only 1 match per key
  'update the map to contain only 1 match per key'.format(k, len(v)))
/usr/local/lib/python3.7/dist-packages/openclean/data/mapping.py:223: UserWarning: Ignoring key: BRONX (1599801 matches). To include ignored keys, update the map to contain only 1 match per key
  'update the map to contain only 1 match per key'.format(k, len(v)))
/usr/local/lib/python3.7/dist-packages/openclean/data/mapping.py:223: UserWarning: Ignoring key: QUEENS (1463554 matches). To include ignored keys, update the map to contain only 1 match per key
  'update the map to contain only 1 match per key'.format(k, len(v)))
/usr/local/lib/python3.7/dist-packages/openclean/data/mapping.py:223: UserWarning: Ignoring key: BROOKLYN (2186681 matches). To include ignored keys, update the map to contain only 1 match per key
  'upd

['MANHATTAN' 'BRONX' 'QUEENS' 'BROOKLYN' 'STATEN ISLAND']


### The above script can be used to fix any error in the spellings of Borough names or for any other column by specifying it in the vocabulary list.

### There are null values present which can be removed before running this script on the data.

#### Dropping Rows where Borough Name is NULL

In [ ]:
df.count()

7364664

In [ ]:
df.filter(df.BORO_NM.isNull()).show()

+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-----------+------------+-------------+--------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|RPT_DT|KY_CD|OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|LOC_OF_OCCUR_DESC|PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+------+-----+---------+-----+-------+----------------+----------+-------+-----------------+-------------+----------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+-------

#### Check if all Rows of the column 'CRM_ATPT_CPTD_CD' have the expected values. Either Completed or Atempted. From the statistics above, there are only 7 rows that have null value for this column. Since it constitutes of very less percentage of the entire dataset, we decided to drop the rows that have null values for this column.

In [ ]:
df.select('CRM_ATPT_CPTD_CD').distinct().show()

+----------------+
|CRM_ATPT_CPTD_CD|
+----------------+
|       ATTEMPTED|
|       COMPLETED|
|            null|
+----------------+



In [ ]:
# similar to pandas' value_counts
df.groupBy('CRM_ATPT_CPTD_CD').count().orderBy('count', ascending=False).show()

+----------------+-------+
|CRM_ATPT_CPTD_CD|  count|
+----------------+-------+
|       COMPLETED|7239063|
|       ATTEMPTED| 125594|
|            null|      7|
+----------------+-------+



In [ ]:
df.filter(df.CRM_ATPT_CPTD_CD.isNull()).show()

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------+--------+----------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+--------

In [ ]:
df = df.filter(df.CRM_ATPT_CPTD_CD.isNotNull())

In [ ]:
df.count()

7364657

#### We can see that all the key codes are valid 3-digit numbers

In [ ]:
df.filter((df.KY_CD < 100) | (df.KY_CD > 999)).count()

0

## Defining checks for outliers in age group

In [ ]:
df.select('SUSP_AGE_GROUP').distinct().show()

+--------------+
|SUSP_AGE_GROUP|
+--------------+
|            -1|
|           924|
|          -965|
|          2019|
|           <18|
|          -966|
|         25-44|
|          null|
|           -64|
|          -973|
|       UNKNOWN|
|           -80|
|           929|
|           65+|
|          -968|
|           930|
|         18-24|
|          -964|
|         45-64|
|           947|
+--------------+
only showing top 20 rows



#### There are many invalid age groups like negative values, unrealistically high age groups, etc.

#### Lets find all the invalid age groups and replace them with `NaN`

In [ ]:
valid_age_groups = ['<18','18-24','25-44','45-64','65+',np.NaN]
df = df.withColumn('SUSP_AGE_GROUP', when(df.SUSP_AGE_GROUP.isin(valid_age_groups), df.SUSP_AGE_GROUP).otherwise(np.NaN))
df.show()

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------+--------+----------------+--------------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+----

In [ ]:
df.select('SUSP_AGE_GROUP').distinct().show()

+--------------+
|SUSP_AGE_GROUP|
+--------------+
|           <18|
|         25-44|
|           65+|
|           NaN|
|         18-24|
|         45-64|
+--------------+



In [ ]:
df.show(n=5)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+----------------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+--------------------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+

In [ ]:
df.select('VIC_AGE_GROUP').distinct().show()

+-------------+
|VIC_AGE_GROUP|
+-------------+
|          -71|
|         -942|
|          938|
|         -954|
|          -39|
|          951|
|          <18|
|         1013|
|        25-44|
|         null|
|         -946|
|         -967|
|          -50|
|         -961|
|          -56|
|          943|
|          -67|
|          936|
|          927|
|          941|
+-------------+
only showing top 20 rows



#### There are many invalid age groups like negative values, unrealistically high age groups, etc.

#### Lets find all the invalid age groups and replace them with `NaN`

In [ ]:
valid_age_groups = ['<18','18-24','25-44','45-64','65+',np.NaN]
df = df.withColumn('VIC_AGE_GROUP', when(df.VIC_AGE_GROUP.isin(valid_age_groups), df.VIC_AGE_GROUP).otherwise(np.NaN))
df.show()

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+-------------------+-----------------+--------+----------+-----------+--------------+--------------+--------+----------------+--------------------+--------------------+------------+-------------+--------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|     SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|             Lat_Lon|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|      VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+----

In [ ]:
df.select('VIC_AGE_GROUP').distinct().show()

+-------------+
|VIC_AGE_GROUP|
+-------------+
|          <18|
|        25-44|
|          65+|
|          NaN|
|        18-24|
|        45-64|
+-------------+



In [ ]:
df.show(n=5)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+---------+-----------------+--------------------+----------------+-----------------+--------+----------+-----------+--------------+---------+--------+----------------+--------------------+------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|  BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|      JURIS_DESC|JURISDICTION_CODE|PARKS_NM|HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         PATROL_BORO|STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+

### Check for Race Values 

In [ ]:
df.select('SUSP_RACE').distinct().show()

+--------------------+
|           SUSP_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|                null|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|             UNKNOWN|
|ASIAN / PACIFIC I...|
|               OTHER|
+--------------------+



#### Replace all `UNKNOWN` values with `NaN`

In [ ]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("SUSP_RACE",
  regexp_replace("SUSP_RACE", "UNKNOWN", "NaN"))

In [ ]:
df.show(100)

+----------+------------+------------+------------+------------+-----------+----------+-----+--------------------+-----+--------------------+----------------+-----------+-------------+-----------------+--------------------+-------------------+-----------------+--------+-----------------+-----------+--------------+--------------------+--------+----------------+--------------------+--------------------+-------------+--------------------+-------+
|CMPLNT_NUM|CMPLNT_FR_DT|CMPLNT_FR_TM|CMPLNT_TO_DT|CMPLNT_TO_TM|ADDR_PCT_CD|    RPT_DT|KY_CD|           OFNS_DESC|PD_CD|             PD_DESC|CRM_ATPT_CPTD_CD| LAW_CAT_CD|      BORO_NM|LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|         JURIS_DESC|JURISDICTION_CODE|PARKS_NM|       HADEVELOPT|HOUSING_PSA|SUSP_AGE_GROUP|           SUSP_RACE|SUSP_SEX|TRANSIT_DISTRICT|         PATROL_BORO|        STATION_NAME|VIC_AGE_GROUP|            VIC_RACE|VIC_SEX|
+----------+------------+------------+------------+------------+-----------+----------+-----+-----------

In [ ]:
df.select('SUSP_RACE').distinct().show()

+--------------------+
|           SUSP_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|                null|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|                 NaN|
|ASIAN / PACIFIC I...|
|               OTHER|
+--------------------+



In [ ]:
df.select('VIC_RACE').distinct().show()

+--------------------+
|            VIC_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|                null|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|             UNKNOWN|
|ASIAN / PACIFIC I...|
|               OTHER|
+--------------------+



In [ ]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("VIC_RACE",
  regexp_replace("VIC_RACE", "UNKNOWN", "NaN"))

In [ ]:
df.select('VIC_RACE').distinct().show()

+--------------------+
|            VIC_RACE|
+--------------------+
|               WHITE|
|               BLACK|
|AMERICAN INDIAN/A...|
|                null|
|      BLACK HISPANIC|
|      WHITE HISPANIC|
|                 NaN|
|ASIAN / PACIFIC I...|
|               OTHER|
+--------------------+



### Checks for Suspect & Victim Sex

In [ ]:
df.select('SUSP_SEX').distinct().show()

+--------+
|SUSP_SEX|
+--------+
|       F|
|    null|
|       M|
|       U|
+--------+



#### Checking values in suspect sex

In [ ]:
df.groupBy('SUSP_SEX').count().orderBy('count', ascending=False).show()

+--------+-------+
|SUSP_SEX|  count|
+--------+-------+
|    null|3554679|
|       M|2390246|
|       F| 755676|
|       U| 664056|
+--------+-------+



In [ ]:
df.select('VIC_SEX').distinct().show()

+-------+
|VIC_SEX|
+-------+
|      F|
|   null|
|      E|
|      M|
|      D|
+-------+



In [ ]:
df.groupBy('VIC_SEX').count().orderBy('count', ascending=False).show()

+-------+-------+
|VIC_SEX|  count|
+-------+-------+
|      F|2884877|
|      M|2427980|
|      E|1149967|
|      D| 901528|
|   null|    305|
+-------+-------+



#### Get unique values of offense description in sorted order

In [ ]:
df.select('OFNS_DESC').distinct().orderBy('OFNS_DESC', ascending=True).show()

+--------------------+
|           OFNS_DESC|
+--------------------+
|                null|
|            ABORTION|
| ADMINISTRATIVE CODE|
|ADMINISTRATIVE CODES|
|AGRICULTURE & MRK...|
|ALCOHOLIC BEVERAG...|
|ANTICIPATORY OFFE...|
|               ARSON|
|ASSAULT 3 & RELAT...|
|     BURGLAR'S TOOLS|
|            BURGLARY|
|CHILD ABANDONMENT...|
|CRIMINAL MISCHIEF...|
|   CRIMINAL TRESPASS|
|     DANGEROUS DRUGS|
|   DANGEROUS WEAPONS|
|  DISORDERLY CONDUCT|
|DISRUPTION OF A R...|
|ENDAN WELFARE INCOMP|
|            ESCAPE 3|
+--------------------+
only showing top 20 rows



#### Getting Total Count of Offense Description

In [ ]:
df.select('OFNS_DESC').distinct().count()

71

In [ ]:
df.groupBy('OFNS_DESC').count().show()

+--------------------+------+
|           OFNS_DESC| count|
+--------------------+------+
|OTHER TRAFFIC INF...|    15|
|ANTICIPATORY OFFE...|   128|
|   FELONY SEX CRIMES|    18|
|NEW YORK CITY HEA...|    93|
|OTHER OFFENSES RE...| 16226|
|VEHICLE AND TRAFF...| 91734|
|KIDNAPPING & RELA...|  2874|
|HOMICIDE-NEGLIGEN...|   104|
|OFF. AGNST PUB OR...|383063|
|PETIT LARCENY OF ...|  1334|
|      FELONY ASSAULT|285879|
|ALCOHOLIC BEVERAG...|  1277|
|OFFENSES RELATED ...|  1691|
|CRIMINAL MISCHIEF...|744482|
|         THEFT-FRAUD| 76591|
|   THEFT OF SERVICES|  4527|
|            JOSTLING|   446|
|MISCELLANEOUS PEN...|185656|
|LOITERING/GAMBLIN...|   245|
|               ARSON| 17639|
+--------------------+------+
only showing top 20 rows



**Map Key Codes with Offense Description**

In [ ]:
key_off_mapping = df.groupBy('KY_CD').agg(collect_set('OFNS_DESC').alias('OFNS_DESCS')).orderBy('KY_CD')
key_off_mapping.show()

+-----+--------------------+
|KY_CD|          OFNS_DESCS|
+-----+--------------------+
|  102|[HOMICIDE-NEGLIGE...|
|  103|[HOMICIDE-NEGLIGE...|
|  104|              [RAPE]|
|  105|           [ROBBERY]|
|  106|    [FELONY ASSAULT]|
|  107|          [BURGLARY]|
|  109|     [GRAND LARCENY]|
|  110|[GRAND LARCENY OF...|
|  111|[POSSESSION OF ST...|
|  112|       [THEFT-FRAUD]|
|  113|           [FORGERY]|
|  114|             [ARSON]|
|  115|[PROSTITUTION & R...|
|  116|[SEX CRIMES, FELO...|
|  117|   [DANGEROUS DRUGS]|
|  118| [DANGEROUS WEAPONS]|
|  119|[INTOXICATED/IMPA...|
|  120|[ENDAN WELFARE IN...|
|  121|[CRIMINAL MISCHIE...|
|  122|          [GAMBLING]|
+-----+--------------------+
only showing top 20 rows



In [ ]:
key_off_mapping.count()

73

In [ ]:
df.select('KY_CD').distinct().count()

73

### Functional Dependency Violation for OFNS_DESC & KY_CD

In [ ]:
ofns_key = ds.to_df()

In [ ]:
from openclean.operator.map.violations import fd_violations
from openclean.operator.collector.count import distinct

fd2_violations = fd_violations(ofns_key, ['KY_CD'], ['OFNS_DESC'])

print('# of violations for FD(KY_CD -> OFNS_DESC) is {}\n'.format(len(fd2_violations)))
for key, gr in fd2_violations.items():
    print(gr[['KY_CD', 'OFNS_DESC']])

# of violations for FD(KY_CD -> OFNS_DESC) is 9

       KY_CD                OFNS_DESC
104      126  MISCELLANEOUS PENAL LAW
161      126  MISCELLANEOUS PENAL LAW
195      126  MISCELLANEOUS PENAL LAW
243      126  MISCELLANEOUS PENAL LAW
244      126  MISCELLANEOUS PENAL LAW
...      ...                      ...
323640   126  MISCELLANEOUS PENAL LAW
323643   126  MISCELLANEOUS PENAL LAW
323665   126  MISCELLANEOUS PENAL LAW
323723   126  MISCELLANEOUS PENAL LAW
323762   126  MISCELLANEOUS PENAL LAW

[10450 rows x 2 columns]
       KY_CD                       OFNS_DESC
119      343  OTHER OFFENSES RELATED TO THEF
629      343  OTHER OFFENSES RELATED TO THEF
1167     343  OTHER OFFENSES RELATED TO THEF
1877     343               THEFT OF SERVICES
2605     343  OTHER OFFENSES RELATED TO THEF
...      ...                             ...
320888   343  OTHER OFFENSES RELATED TO THEF
322746   343  OTHER OFFENSES RELATED TO THEF
323015   343  OTHER OFFENSES RELATED TO THEF
323021   343  OTHER

In [ ]:
from openclean.operator.collector.repair import Shortest, Vote, conflict_repair

# Define the conflict resolution strategy. We use a majority vote for RHS attributes.
strategy = {'KY_CD': Vote()}

# resolve the conflicts
resolved = conflict_repair(conflicts=fd1_violations, strategy=strategy, in_order=False)

In [ ]:
#resolving for key code -> offense description

from openclean.operator.collector.repair import Shortest, Vote, conflict_repair

# Define the conflict resolution strategy. We use a majority vote for RHS attributes.
strategy = {'OFNS_DESC': Vote(tiebreaker=Shortest())}

# resolve the conflicts
resolved = conflict_repair(conflicts=fd2_violations, strategy=strategy, in_order=False)

In [ ]:
trial = fd_violations(resolved, ['KY_CD'], ['OFNS_DESC'])

print('# of violations for FD(KY_CD -> OFNS_DESC)) is {}\n'.format(len(trial)))
print(violation_group)

# of violations for FD(KY_CD -> OFNS_DESC)) is 0

       CMPLNT_NUM  ...                       New Georeferenced Column
13      276278658  ...  POINT (-73.94569741099998 40.786182447000044)
166     890663602  ...   POINT (-73.91843446099993 40.80893116500005)
180     728758084  ...  POINT (-73.91212195899993 40.747301667000045)
181     985806516  ...  POINT (-73.98411999099994 40.754849215000036)
268     842358698  ...   POINT (-73.85902567999993 40.88735603500004)
...           ...  ...                                            ...
323571  172047038  ...   POINT (-73.90410359499998 40.66646466200007)
323678  217581853  ...   POINT (-73.89515414499994 40.66132136900006)
323733  793541563  ...  POINT (-73.80633674899997 40.701804167000034)
323738  306924169  ...   POINT (-73.89690223799995 40.66833311800008)
323806  476968757  ...   POINT (-73.87565666399996 40.75686097700003)

[4058 rows x 36 columns]


In [ ]:
violation_group = resolved[(resolved['KY_CD']=='125')]

fd3_violations = fd_violations(resolved, ['KY_CD'], ['OFNS_DESC'])

print('# of violations for FD(KY_CD -> OFNS_DESC) is {}\n'.format(len(fd3_violations)))
print(violation_group)

# of violations for FD(KY_CD -> OFNS_DESC) is 0

       CMPLNT_NUM  ...                       New Georeferenced Column
1307    243028990  ...   POINT (-74.00125319299997 40.71634415200003)
2550    726416926  ...   POINT (-73.77623390699993 40.67998073800004)
3208    533129014  ...   POINT (-73.89776780399995 40.86326000900005)
3579    657165465  ...  POINT (-73.99398802999998 40.716193040000064)
5803    822041811  ...   POINT (-73.92894705999998 40.61544337400005)
...           ...  ...                                            ...
317623  926822345  ...   POINT (-74.00125319299997 40.71634415200003)
319353  486010364  ...   POINT (-73.91354311599997 40.75347568500007)
320550  398377413  ...   POINT (-74.13004566699995 40.63852488600002)
320592  777788721  ...   POINT (-74.00125319299997 40.71634415200003)
322429  999058929  ...   POINT (-73.94184447299993 40.75709455700007)

[307 rows x 36 columns]


In [ ]:
resolved.groupby(['OFNS_DESC', 'KY_CD']).size()

OFNS_DESC                       KY_CD
ADMINISTRATIVE CODE             365       528
                                675        34
ALCOHOLIC BEVERAGE CONTROL LAW  346        52
ANTICIPATORY OFFENSES           354        13
ARSON                           114       472
                                         ... 
SEX CRIMES                      116      1117
                                233      4046
THEFT-FRAUD                     112      3003
UNAUTHORIZED USE OF A VEHICLE   353       936
VEHICLE AND TRAFFIC LAWS        348      6060
Length: 63, dtype: int64

In [ ]:
keymapping = resolved.groupby('KY_CD').apply(lambda x: x['OFNS_DESC'].unique())
print(keymapping)

KY_CD
101    [MURDER & NON-NEGL. MANSLAUGHTER]
102         [HOMICIDE-NEGLIGENT-VEHICLE]
103     [HOMICIDE-NEGLIGENT,UNCLASSIFIE]
104                               [RAPE]
105                            [ROBBERY]
                     ...                
572                 [DISORDERLY CONDUCT]
578                      [HARRASSMENT 2]
675                [ADMINISTRATIVE CODE]
677    [NYS LAWS-UNCLASSIFIED VIOLATION]
678            [MISCELLANEOUS PENAL LAW]
Length: 63, dtype: object


#### Each key code represents a particular offense description. There is a one to one mapping. So we would use key code for future analysis instead of offense description.

#### Calculating the null values present in the data columnwise (with respect to the features)

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [ ]:
amount_missing_df = df.select([(count(when(isnan(c) | col(c).isNull(), c))/count(lit(1))).alias(c) for c in df.columns])
amount_missing_df.show()

+----------+--------------------+-------------------+------------------+-------------------+--------------------+------+-----+--------------------+-----+-------+----------------+----------+-------+-------------------+--------------------+----------+-----------------+-------------------+------------------+-------------------+------------------+------------------+-----------------+------------------+--------------------+------------------+-------------------+----------------+--------------------+
|CMPLNT_NUM|        CMPLNT_FR_DT|       CMPLNT_FR_TM|      CMPLNT_TO_DT|       CMPLNT_TO_TM|         ADDR_PCT_CD|RPT_DT|KY_CD|           OFNS_DESC|PD_CD|PD_DESC|CRM_ATPT_CPTD_CD|LAW_CAT_CD|BORO_NM|  LOC_OF_OCCUR_DESC|       PREM_TYP_DESC|JURIS_DESC|JURISDICTION_CODE|           PARKS_NM|        HADEVELOPT|        HOUSING_PSA|    SUSP_AGE_GROUP|         SUSP_RACE|         SUSP_SEX|  TRANSIT_DISTRICT|         PATROL_BORO|      STATION_NAME|      VIC_AGE_GROUP|        VIC_RACE|             VIC_SEX|


#### Thus, we can see that the percentage of null values per variable has gone considerably down after cleaning. Some variables like 'PARKS_NM', 'HADEVELOPT' and such can have null values as established above. 

JURISDICTION wise count

In [ ]:
df.groupBy('JURIS_DESC').count().show()

+--------------------+-------+
|          JURIS_DESC|  count|
+--------------------+-------+
|    N.Y. POLICE DEPT|6546928|
|      PORT AUTHORITY|  34227|
| N.Y. TRANSIT POLICE| 163584|
|  HEALTH & HOSP CORP|   3690|
|    U.S. PARK POLICE|    282|
|   N.Y. STATE POLICE|   1883|
| N.Y. HOUSING POLICE| 561186|
| TRI-BORO BRDG TUNNL|   5670|
|         METRO NORTH|    822|
|    N.Y. STATE PARKS|    455|
|NEW YORK CITY SHE...|    279|
| DEPT OF CORRECTIONS|   9638|
|NYS DEPT TAX AND ...|    105|
|               OTHER|  24123|
|FIRE DEPT (FIRE M...|    477|
|  LONG ISLAND RAILRD|    546|
|     POLICE DEPT NYC|   8948|
|             AMTRACK|    188|
|             CONRAIL|     16|
|STATN IS RAPID TRANS|    439|
+--------------------+-------+
only showing top 20 rows



### Number of columns in Clean Data

In [ ]:
len(df.columns)

30

### Number of rows in Clean Data

In [ ]:
df.count()

7364657

In [ ]:
df.printSchema()

root
 |-- CMPLNT_NUM: integer (nullable = true)
 |-- CMPLNT_FR_DT: string (nullable = true)
 |-- CMPLNT_FR_TM: string (nullable = true)
 |-- CMPLNT_TO_DT: string (nullable = true)
 |-- CMPLNT_TO_TM: string (nullable = true)
 |-- ADDR_PCT_CD: integer (nullable = true)
 |-- RPT_DT: string (nullable = true)
 |-- KY_CD: integer (nullable = true)
 |-- OFNS_DESC: string (nullable = true)
 |-- PD_CD: integer (nullable = true)
 |-- PD_DESC: string (nullable = true)
 |-- CRM_ATPT_CPTD_CD: string (nullable = true)
 |-- LAW_CAT_CD: string (nullable = true)
 |-- BORO_NM: string (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PREM_TYP_DESC: string (nullable = true)
 |-- JURIS_DESC: string (nullable = true)
 |-- JURISDICTION_CODE: integer (nullable = true)
 |-- PARKS_NM: string (nullable = true)
 |-- HADEVELOPT: string (nullable = true)
 |-- HOUSING_PSA: string (nullable = true)
 |-- SUSP_AGE_GROUP: string (nullable = true)
 |-- SUSP_RACE: string (nullable = true)
 |-- SUSP_S

### **Exporting Clean Data in CSV**

The Cleaned Data Set will be saved as `NYPD_Complaint_Data_Historic_Cleaned.csv`

In [ ]:
pd_df = df.toPandas()
pd_df.to_csv("NYPD_Complaint_Data_Historic_Cleaned_Spark.csv")